# Deep neural network classifying cats

In this notebook we will try to train a model to identify cats in images.

In [6]:
import numpy as np
import matplotlib.pyplot as plt

from utils.data_utils import load_cat_data

from deep_nn import DeepNNModel